In [47]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt
import pickle

In [53]:
def apply_Pauli_noise(wire, noise_probability, noise_strength):
    if np.random.rand() < noise_probability:
        qml.RX(noise_strength, wires=wire)
    if np.random.rand() < noise_probability: 
        qml.RY(noise_strength, wires=wire)
    if np.random.rand() < noise_probability: 
        qml.RZ(noise_strength, wires=wire)  
            
def create_Ising_Hamiltonian(h,number_wires):
    couplings = [-h]
    ops = [qml.PauliX(0)]

    for i in range(number_wires-1):
        couplings = [-h] + couplings
        ops = [qml.PauliX(i+1)] + ops   
    
    for i in range(number_wires):
        couplings = [-1] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

    return qml.Hamiltonian(couplings,ops)

def create_Heisenberg_Hamiltonian(Jx,Jy,Jz,h,number_wires):
    couplings = [-Jx]*number_wires+[-Jy]*number_wires+[-Jz]*number_wires+[-h]*number_wires
    
    ops=[qml.PauliX(0)@qml.PauliX(1)]
    for i in range(number_wires-1):
        ops=ops+[qml.PauliX(i+1)@qml.PauliX((i+2)%number_wires)]
    for i in range(number_wires):
        ops=ops+[qml.PauliY(i)@qml.PauliY((i+1)%number_wires)]
    for i in range(number_wires):
        ops=ops+[qml.PauliZ(i)@qml.PauliZ((i+1)%number_wires)]
    for i in range(number_wires):
        ops=ops+[qml.PauliZ(i)]
    return qml.Hamiltonian(couplings,ops)

backend="default.mixed"
number_wires=4
dev = qml.device(backend, wires=number_wires)
@qml.qnode(dev)

def model(params,H,p,s,number_wires):
    for i in range(number_wires):
        qml.RY(params[i], wires=i)
    for i in range(number_wires):
        qml.CNOT([i,(i+1)%number_wires])
        apply_Pauli_noise(i,p,s)
        apply_Pauli_noise((i+1)%number_wires,p,s)
    for i in range(number_wires):
        qml.RZ(params[i+number_wires], wires=i)
    for i in range(number_wires):
        qml.RY(params[i+2*number_wires], wires=i)
    
    for i in range(number_wires):
        qml.CNOT([i,(i+1)%number_wires])
        apply_Pauli_noise(i,p,s)
        apply_Pauli_noise((i+1)%number_wires,p,s)
    for i in range(number_wires):
        qml.RZ(params[i+3*number_wires], wires=i)
    return qml.expval(H)

def train(Jx,Jy,Jz,h,p,s,number_wires):
    return(3,[0,1,2,3])
    step_size = 0.02
    opt = qml.GradientDescentOptimizer(stepsize=0.2)
    H =  create_Heisenberg_Hamiltonian(Jx,Jy,Jz,h,number_wires)
    theta = np.array([np.pi/2 for i in range(4*number_wires)], requires_grad=True)
    energy = [model(theta, H,p,s,number_wires)]
    params = [theta]
    max_iterations = 500
    conv_tol = 1e-06
    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(model, theta, H=H,p=p,s=s,number_wires=number_wires)
        energy.append(model(theta, H,p,s,number_wires))
        params.append(theta)
        conv = np.abs(energy[-1] - prev_energy)
        if n % 10 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
        if conv <= conv_tol:
            break
    return theta,energy

In [ ]:
s=0.2
Jx=1
Jy=2
Jz=1
h=0.5
ps=[0,0.01,0.02,0.05,0.1,0.2,0.5]
energies=[]
for p in ps:
    print("p =",p)
    theta,energy=train(Jx,Jy,Jz,h,p,s,number_wires)
    energies.append(energy)

In [ ]:
for i,energy in enumerate(energies):
    plt.plot(np.array(energy),label="p={}".format(ps[i]))
plt.title("convergence of ground state finding for Heisenberg model with noise strength={}".format(s))
plt.xlabel("step")
plt.ylabel("energy")
plt.xscale("log")
plt.legend()

In [ ]:
file_name="energies.pkl"
open_file = open(file_name, "wb")
pickle.dump(energies, open_file)
open_file.close()